# **Plate Number Recognition System using Robinsons Compass Mask Edge Detection | Test Document** 
**IMAGPRO**

ALONZO, Jose Anton S.

HERNANDEZ, Pierre Vincent C.

SOLIS, Frances Danielle B.

SUSADA, Stephanie R.

### *Import necessary header files or libraries*

In [1]:
import rcm
import numpy as np
import glob
import cv2 as cv
import matplotlib.pyplot as plt

import string
import easyocr

import video

#from ultralytics import YOLO

## ***PHASE 1: Edge Detection***

In [2]:
vid = cv.VideoCapture(1)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
#     cv.imshow('frame', frame)
    print(in)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
vid.release()
cv.destroyAllWindows

First, read the image data and convert it to grayscale in preparation for the edge detection process

In [ ]:
    # !!! CONFIGURE IMAGE FILE PARAMETERS HERE !!!
    img_name = "plate1"
    img_ext = ".png"
    img_path = "input/" + img_name + img_ext

    # Get grayscale version of the image
    img_raw = cv.imread(img_path)
    img_data = cv.cvtColor(img_raw, cv.COLOR_BGR2GRAY)
    plt.imshow(img_data, cmap='gray')


In [ ]:
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
vid.release()
cv.destroyAllWindows

In [ ]:
rcm.createDirectories(img_name)

Finalize the edge detection process

In [ ]:
edged_img = rcm.detectEdgeAbs(img_data, img_name)
plt.imshow(edged_img, cmap="gray")

In [ ]:
edged_img = rcm.detectEdgeMax(img_data, img_name)
plt.imshow(edged_img, cmap="gray")

## ***PHASE 2: Process Images***

In [ ]:
# Initialize the OCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Mapping dictionaries for character conversion
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}
dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

In [ ]:
def license_complies_format(text):
    """
    Check if the license plate text complies with the required format.

    Args:
        text (str): License plate text.

    Returns:
        bool: True if the license plate complies with the format, False otherwise.
    """
    if len(text) != 6:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()):
        return True
    else:
        return False

In [ ]:
def format_license(text):
    """
    Format the license plate text by converting characters using the mapping dictionaries.

    Args:
        text (str): License plate text.

    Returns:
        str: Formatted license plate text.
    """
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 2: dict_int_to_char,
               3: dict_char_to_int, 4: dict_char_to_int, 5: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_


In [ ]:
def read_license_plate(license_plate_crop):
    """
    Read the license plate text from the given cropped image.

    Args:
        license_plate_crop (PIL.Image.Image): Cropped image containing the license plate.

    Returns:
        tuple: Tuple containing the formatted license plate text and its confidence score.
    """

    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection
        
        if len(text) == 7:
            text = text[:3] + text[4:]

        text = text.upper().replace(' ', '')

        if license_complies_format(text):
            return format_license(text), score

    return None, None

In [ ]:
license_plate_text, license_plate_text_score = read_license_plate(edged_img)

# results = {}

# if license_plate_text is not None:
#     results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
#                                   'license_plate': {'bbox': [x1, y1, x2, y2],
#                                                     'text': license_plate_text,
#                                                     'bbox_score': score,
#                                                     'text_score': license_plate_text_score}}

In [ ]:
print(license_plate_text)